# Loading data
Data (videos, tracks, poses) is organized in a specified folder structure on the lab volume. 
To simplify loading data, a config file describes where each type of data is found within this structure this. For instance, the movies are always in the `dat` or the `dat.processed` folder. Poses in `res`. Within each folder each experiment has a unique identifier composed of the name of the recording rig and the date nad time of the experiment. In addition, the config file also has information about how to load each file. More information on the structure of the config file and how to modify it [below](#Config-files).

Currently supported types are `poses`, `tracks`, `video`. Planned: `timestamps`, `daq`.

## Usage
The `DataLoader` uses all this information to load all described data types give this identifier:
Initialize data loader - this will load the config file.

In [1]:
from leap_utils.io import DataLoader

config_file = 'io.yaml'
root_dir = '/Volumes/ukme04/#Common/chainingmic'  # this should point to the lab volume mount
dl = DataLoader(config_file, root_dir)

Specific data files from an experiment `expt_id` can be loaded using the `get` function:

In [6]:
expt_id = 'localhost-20180618_151056'
print(dl.path('video', expt_id))  # get constructe file name
vr = dl.get('video', expt_id)  # returns a `Videoreader`
tracks = dl.get('tracks', expt_id)  # returns a dict loaded via deepdish
print(tracks.keys())
poses = dl.get('poses', expt_id)  # same
print(poses.keys())


/Volumes/ukme04/#Common/chainingmic/dat.processed/localhost-20180618_151056/localhost-20180618_151056.mp4
dict_keys(['background', 'centers', 'centers_initial', 'chambers', 'chambers_bounding_box', 'frame_error', 'frame_error_codes', 'led', 'led_coords', 'led_mask', 'lines', 'file_name', 'frame_count', 'nchambers', 'nflies', 'nframes', 'start_frame', 'status', 'threshold'])
dict_keys(['bad_boxes', 'confidence', 'fixed_angles', 'fly_frame', 'fly_id', 'frame_numbers', 'positions', 'expID', 'last_saved_batch'])


## Config files
A config file consists of three sections: `root`, `types`, and `loaders`.
`root` specifies the default root directory - can be overriden when creating the `DataLoader`.
```yaml
root: /Volumes/ukme04/#Common/chainingmic
```

### File types
`types` defines all file types. Definitions contain 
- the base folder in the root directory, 
- the suffix. The file path is constructed from the folder and the suffix as `rootdir/folder/ID/IDsuffix`. This scheme cold easily be made another parameter.
- a loader (any of the sections defined in `loaders`, see next paragraph). Optional but recommended - if missing, the loader is inferred from the suffix.

```yaml
types:
    poses:
      folder: res
      suffix: _poses.h5
      loader: hdf5
    video:
      folder: [dat, dat.processed]
      suffix: .mp4
      loader: video
    tracks:
      folder: res
      suffix: [_tracks_fixed.h5, _tracks.h5]
      loader: hdf5
```

### File loaders
`loaders` defines how different file types are read. Definitions contain 
- `extensions`: Used to match file names with loaders if not explicitely specified in the types definition. Based on the filenames ending in any of the extensions.
- `module`: automatically imported
- `function`: function in `module` to use a loader (see next section).
- `args`, `kwargs`: Optional arguments to be passed to the function. This information is used create a command for loading that looks like this: `module.function(filepath, *args, **kwargs).

```yaml
loaders:
    hdf5:
        extensions: [h5, hdf, hdf5]
        module: deepdish
        function: io.load
    video:
        extensions: [mp4, avi]
        module: videoreader
        function: VideoReader
    npy:
        extensions: [npy, npz]
        module: numpy
        function: load
    csv:
        extensions: [csv]
        module: pandas
        function: from_csv
        # args: [arguments]
        # kwargs: {Delimiter: \t}
```

